In [1]:
import spacy
import torch
from torchtext import data    
from libs import sql_tokenizer
import libs

In [2]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [3]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [4]:
fields = [(None, None), ('text',TEXT),('label', LABEL)]

In [ ]:
training_data=data.TabularDataset(path = 'csv_files/xss.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))
for i,dt in enumerate(training_data):
    if(len(dt.text) <= 0):
        training_data[i].text = "<blank>"
        print(i,training_data[i].text,training_data[i].label)

{'text': ['LT', 'script', 'GT', 'alert', 'LPRN', 'INT', 'RPRN', 'SMCLN', 'LT', 'SLSH', 'script', 'GT'], 'label': 'xss'}
19 <blank> xss
20 <blank> xss
22 <blank> xss
24 <blank> xss
26 <blank> xss
28 <blank> xss
30 <blank> xss
32 <blank> xss
34 <blank> xss
36 <blank> xss
38 <blank> xss
40 <blank> xss
42 <blank> xss
44 <blank> xss
46 <blank> xss
48 <blank> xss
50 <blank> xss
52 <blank> xss
55 <blank> xss
57 <blank> xss
61 <blank> xss
63 <blank> xss
65 <blank> xss
67 <blank> xss
69 <blank> xss
71 <blank> xss
73 <blank> xss
75 <blank> xss
77 <blank> xss
79 <blank> xss
81 <blank> xss
83 <blank> xss
85 <blank> xss
87 <blank> xss
89 <blank> xss
91 <blank> xss
93 <blank> xss
95 <blank> xss
97 <blank> xss
99 <blank> xss
101 <blank> xss
103 <blank> xss
105 <blank> xss
107 <blank> xss
109 <blank> xss
111 <blank> xss
113 <blank> xss
115 <blank> xss
117 <blank> xss
119 <blank> xss
121 <blank> xss
123 <blank> xss
125 <blank> xss
127 <blank> xss
129 <blank> xss
131 <blank> xss
133 <blank> xss
135 <bla

In [ ]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [ ]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
#print(TEXT.vocab.freqs.most_common(100))  

#Word dictionary
print(TEXT.vocab.stoi)   

Size of TEXT vocabulary: 3499
Size of LABEL vocabulary: 4


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x00000220B4823A58>>, {'<unk>': 0, '<pad>': 1, 'DOT': 2, 'the': 3, 'A': 4, 'AND': 5, 'OF': 6, 'TO': 7, 'INT': 8, 'br': 9, 'IS': 10, 'IN': 11, 'it': 12, 'i': 13, 'RPRN': 14, 'this': 15, 'CMMA': 16, 'that': 17, 'BITAND': 18, 'SMCLN': 19, 's': 20, 'SQUT': 21, 'LPRN': 22, 'NULL': 23, 'AS': 24, 'movie': 25, 'was': 26, 'WITH': 27, 'FOR': 28, 'SLSH': 29, 'but': 30, 'film': 31, 'PRCNT': 32, 'ON': 33, 'EQ': 34, 'you': 35, 'T': 36, 'NOT': 37, 'HASH': 38, 'DQUT': 39, 'his': 40, 'ARE': 41, 'one': 42, 'FROM': 43, 'MINUS': 44, 'be': 45, 'he': 46, 'ALL': 47, 'have': 48, 'an': 49, 'they': 50, 'who': 51, 'LT': 52, 'OR': 53, 'AT': 54, 'GT': 55, 'IF': 56, 'BY': 57, 'BSLSH': 58, 'CLN': 59, 'LIKE': 60, 'so': 61, 'OUT': 62, 'about': 63, 'can': 64, 'has': 65, 'just': 66, 'there': 67, 'story': 68, 'good': 69, 'MORE': 70, 'her': 71, 'TIME': 72, 'WHEN': 73, 'script': 74, 'SOME': 75, 'up': 76, 'which': 77, 'very': 78, 'even': 

In [ ]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.act = nn.Sigmoid()
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,()hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [ ]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [ ]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(3499, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 409,357 trainable parameters
torch.Size([3499, 100])


In [ ]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        
        
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 6
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.626 | Train Acc: 39.80%
	 Val. Loss: 0.428 |  Val. Acc: 32.42%
	Train Loss: 0.052 | Train Acc: 33.31%
	 Val. Loss: -0.309 |  Val. Acc: 31.25%
	Train Loss: -0.793 | Train Acc: 44.49%
	 Val. Loss: -1.124 |  Val. Acc: 60.55%
	Train Loss: -1.500 | Train Acc: 59.62%
	 Val. Loss: -1.698 |  Val. Acc: 64.84%
	Train Loss: -2.050 | Train Acc: 64.80%
	 Val. Loss: -2.082 |  Val. Acc: 68.55%


In [ ]:
import spacy

nlp = spacy.load('en')

def predict(model,sentence):
    
    tokenized = [tok.text for tok in nlp.tokenizer(sql_tokenizer(sentence))] # tokenize the sentence
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] # convert to integer sequence
    print(indexed)
    length = [len(indexed)] #compute no. of words
    tensor = torch.LongTensor(indexed).to(device) # convert to tensor
    tensor = tensor.unsqueeze(1).T
    length_tensor = torch.LongTensor(length)
    prediction = model(tensor,length_tensor)
    return prediction.item()
    


# safe text

In [ ]:
predict(model,"im good")

# low level

In [ ]:
predict(model,'hey there <IMG SRC=javascript:alert(&quot;XSS&quot;)>')

# medium level

In [ ]:
predict(model,"<IMG SRC=&#106;&#97;&#118;&#97;&#115;&#99;&#114;&#105;&#112;&#116;&#58;&#97;&#108;&#101;&#114;&#116;&#40;&#39;&#88;&#83;&#83;&#39;&#41;>")

# high level

In [ ]:
predict(model,"IMG SRC=&#x6A&#x61&#x76&#x61&#x73&#x63&#x72&#x69&#x70&#x74&#x3A&#x61&#x6C&#x65&#x72&#x74&#x28&#x27&#x58&#x53&#x53&#x27&#x29")

# sql injection

In [ ]:
predict(model,"SELECT * from table where id = 1 union select 1,2,3")

In [ ]:
predict(model, 'hey im here hey NAake*#() alert whats up how yah doin man @#')